In [48]:
from typing import *

# ___보석 쇼핑___


#### Two Pointer

In [45]:
# gems = ["ZZZ", "YYY", "NNNN", "YYY", "BBB"]
# gems = ["DIA", "RUBY", "RUBY", "DIA", "DIA", "EMERALD", "SAPPHIRE", "DIA"]
# gems = ["X", "X", "X"]
# gems = ["A","B","B","B","C","B","A"]
# gems = ["AA", "AB", "AC", "AA", "AC"]

In [1]:
# 윈도우의 최소 사이즈는 알 수 있지만,최소 사이즈 이상이 필요하다면 결국 윈도우 사이즈를 늘려야하니 비효율적 => 가변의 윈도우 사이즈: 투포인터
# 투포인터 필요 없이 딕셔너리에 [최근에 본 지점, 봤는지의 여부] 넣으면 되겠는데? => 반례로 실패

# 반례 : gems = ["A","B","B","B","C","B","A"]
def wrong_solution(gems):
    window = dict.fromkeys(list(set(gems)), [0, False]) # [last_index, is_seen]
    
    for i, gem in enumerate(gems):
        # window = {'D' = [0, False], 'R': [0, False]}
        # is_seen = [False, False]
        is_seen = [v[1] for v in window.values()]
        if all(is_seen): break
        else:
            window[gem] = [i, True]
    
    last_index = [v[0] for v in window.values()]
    return [min(last_index) + 1, max(last_index) + 1]

In [2]:
# 투 포인터로, right: 투 포인터 범위 내 아직 모든 보석 종류가 포함되어 있지 않으면(not all(dict.values()))
# left: 투 포인터 범위 내 모든 보석 종류가 포함되어 있으면, 언제까지? => 다시 모든 보석 종류가 포함되어 있지 않으면,
# 그때의 left와 right의 범위를 조사하며 업데이트
def solution(gems):
    left, right, end = 0, 0, len(gems)
    section, shortest_section = [left + 1, end], len(gems)
    seen = dict.fromkeys(list(set(gems)), 0)
    
    while right < end:
        # right
        seen[gems[right]] += 1
        
        # left
        if all(seen.values()):
            while left <= right:
                # left를 한번 더 이동해도 될 때
                if seen[gems[left]] > 1:
                    seen[gems[left]] -= 1
                    left += 1
                elif right - left < shortest_section:
                    shortest_section = right - left
                    section = [left + 1, right + 1]
                    break
                else: break # 무한루프 방지

        right += 1
        
    return section

# ___Sliding Window Maximum___


#### Monotonic Queue, Two Pointer

In [62]:
# Brute Force(O(kN)), 
# 시간 복잡도 측면에서는 동일하지만 조금 더 최적화하려면 새로 들어오는 값과 이전 윈도우의 최곳값을 비교하면 계산 횟수를 줄일 수 있다

from collections import deque

# 10000부터 내림차순이면으로 정렬된 nums에서는 윈도우가 이동할 때마다 매번 최댓값을 구해줘야하므로 TLE
def WrongmaxSlidingWindow(nums: List[int], k: int) -> List[int]:
    window, window_max = deque(nums[0:k]), [max(nums[0:k])]
    
    for num in nums[k:]:
        window.append(num)
        
        if window.popletf() == window_max[-1]:
            window_max.append(max(window))
        else:
            if num > window_max[-1]:
                window_max.append(num)
            else:
                window_max.append(window_max[-1])
                
    return window_max

In [78]:
# 윈도우에서 최댓값이 빠지고나면, max()를 계산하는 것이 아니라 -inf로 초기화
# 아, 그래도 10000부터 내림차순이면 같은 말이구나! TLE

import math

def TLEmaxSlidingWindow(nums: List[int], k: int) -> List[int]:
    window = deque(nums[0:k])
    window_max = [max(window)]
    cur_max = window_max[-1]
    
    for num in nums[k:]:
        window.append(num)

        if window.popleft() == cur_max:
            cur_max = -math.inf
        
        # 최댓값이 윈도우에서 빠졌으면(cur_max가 -inf로 초기화되어 있으면) 다시 윈도우의 최댓값을 계산
        if cur_max == -math.inf:
            cur_max = max(window)
        # 최댓값은 윈도우 안에 있어도, 새로 들어온 num이 이전 단계의 최댓값(cur_max)보다 크다면 최댓값을 새로 들어온 num으로 교체
        elif num > cur_max:
            cur_max = num
            
        window_max.append(cur_max)

                
    return window_max

In [83]:
# Monotonic queue : queue가 담고 있는 원소들이 단조 증가/감소하도록 관리
# deque의 front를 maximum으로 하는 monotonic decreasing queue를 구현하면,
# deque를 활용하여 popleft(), pop(), append()모두 양쪽 끝 지점에서 이루어지므로 O(1)에 가능하며

# deque를 이용하면 append(), popleft() => O(1)
# 최댓값은 deque의 front에 항상 저장되어 있음, 이 때 deque는 사실 최댓값의 인덱스를 가지고 있다는 점에 유의
# 투 포인터 이용

from collections import deque

def maxSlidingWindow(nums: List[int], k: int) -> List[int]:
    mdq = deque()
    l = 0
    answer = []
    for r, num in enumerate(nums):
        # 새로 들어올 num보다 작은 값들이 있으면 삭제
        while mdq and nums[mdq[-1]] < num:
            mdq.pop()
        
        mdq.append(r)
    
        # 윈도우 시작점(l)이 현재 mdq front(window 최댓값의 인덱스)를 벗어나면 삭제
        if l > mdq[0]:
            mdq.popleft()
        
        # 처음으로 최소 윈도우 사이즈(k)를 만족하면, 그때부터는 deque front의 value(nums에서의 인덱스, 최댓값)를 저장하며 왼쪽으로 한칸 씩 이동
        if r + 1 >= k:
            answer.append(nums[mdq[0]])
            l += 1
            
    return answer

# ___Longest Repeating Character Replacement___


#### Two Pointer

In [103]:
# 투 포인터로 포인터를 이동하며 Counter.most_common()이외의 Counter들의 합이 K 이하면 합격
# 이외의 Counter의 합 == 현재까지의 문자열 길이에서 most_common()을 뺀 값
# Counter.most_common()

# TLE
from collections import Counter, deque

def TLEcharacterReplacement(s: str, k: int) -> int:
    l = longest = 0
    deq = deque()
    
    for r, c in enumerate(list(s)):
        deq.append(c)
        common = Counter(deq).most_common()[0][1]
        
        # k 이하면 합격
        if (r - l + 1) - common <= k:
            longest = max(longest, r - l + 1)
        else:
            deq.popleft()
            l += 1
    
    return longest

In [ ]:
# TLE 이유를 추론: k 이하인 경우가 많을때?
# 그렇다면 역으로 k 초과시 불합격만 생각한다면

def TLE2characterReplacement(s: str, k: int) -> int:
    l = 0
    deq = deque()
    
    for r, c in enumerate(list(s)):
        deq.append(c)
        common = Counter(deq).most_common()[0][1]
        
        # 문자열 길이 - 가장 많이 등장한 문자 => 교체할 문자의 수
        if (r - l  + 1) - common > k:
            deq.popleft()
            l += 1
    
    # r은 문자열의 끝을 최대한 rear 방향으로, l은 문자열 시작을 최대한 front 방향으로 움직이는데,
    # 한 번 최대 문자열 길이가 정해지고나면, r이 1 움직이면 무조건 l도 1 움직인다 => 그렇다면 r - l이 결국 최대 길이
    return r - l

In [128]:
# 위의 방식 역시 TLE, 그렇다면 deque()를 사용해서일까? 아니면 most_common() => nlogn이여서?
# Counter() 사용안하고 defaultdict로 접근
from collections import defaultdict

def characterReplacement(s: str, k: int) -> int:
    l = 0
    seen = defaultdict(int)
    
    for r, c in enumerate(list(s)):
        seen[c] += 1
        common = max(seen.values())
        
        if (r - l + 1) - common > k:
            seen[s[l]] -= 1
            l += 1
        
    return r - l + 1

In [ ]:
# Discuss 코드

# 1. Counter()를 객체로 만들어 변수로 사용할 수 있구나
# 2. 내가 생각한 방법이랑 비슷한 궤도에서 오히려 result를 계산하는데도 TLE에 걸리지 않은 것으로 봐서는 데크를 사용한게 문제였는 것 같은데
# 3. 오히로 most_common()을 쓰는데도 괜찮은것을 보니 deque()와 Counter() 객체를 사용한 차이인것 같은데
#    deque()는 중간 지점 인덱스에 접근할때는 O(N)으로 보지만, 현재는 끝 지점에서만 관리하기 때문에 O(1)으로 보는게 맞는데 왜 그렇게 뜰까
# 4. 그렇다면 Counter(deuqe()) 이 지점에서 문제일까?, Counter()객체에서 most_common()과 Counter(데크).most_common()에서 시간이 소요되는 것일까

def characterReplacement(self, s, k):
        count = collections.Counter()
        start = result = 0
        for end in range(len(s)):
            count[s[end]] += 1
            max_count = count.most_common(1)[0][1]
            if end - start + 1 - max_count > k:
                count[s[start]] -= 1
                start += 1
            result = max(result, end - start + 1)
        return result